# Assignment 2

Welcome to the second assignment! We're excited to see what you'll create using the techniques you've learned in this course.

First of all, please type your name and UID in the following format:

Firstname Lastname, #UID

In [ ]:
#@title Your Info { display-mode: "form" }

Name = ''  #@param {type:"string"}
UID = ''  #@param {type:"string"}

### Goals
The goals of this assignment are to:

1. Build and train a convolutional neural network from scratch and using built-in PyTorch modules.
2. Build ResNet-18 from scarch and Finetune it different strategies.
3. Visualize the attention of ResNet using Class Activation Mapping
4. Come up with your own model and compete on a test leaderboard for extra credit.

By the end of this assignment, you will have gained experience with:

- Implementing and training cnns using PyTorch.
- Designing you own neural networks with the tips and tricks
- Debugging and troubleshooting issues that may arise during the development process.

Please note that it may take some time to run the entire notebook and prepare the submission version. Make sure to allocate enough time for this task and start early. If you have any questions or run into any issues, please feel free to raise them in the Piazza forum or search the internet for debugging purposes. However, please do not directly copy code from other sources.

This assignment is due on **Sunday, Nov 3**.

**Do not use any Code AI to finish the assignment.**


Good luck and happy coding! Remember, the most important thing is to have fun and learn something new.





## Q0: Data Preparation(0 pts)

As in Assignment 1, we first download the MiniPlaces dataset and upload it to Colab using the following codes.

Note that everytime you open up this notebook, we will need to **re-download and process the dataset**. Don't worry though - this shouldn't take long, usually just a minute or less. Okay, let's get started!

To begin, you will need to download the Miniplaces Dataset from its google drive file ID.

Downloading this dataset typically takes less than 15 seconds.

In [ ]:
# Downloading this file takes about a few seconds.
# Download the tar.gz file from google drive using its file ID.
!pip3 install --upgrade gdown --quiet
# unfortunately, only the following command seems to be constantly working...
!wget https://web.cs.ucla.edu/~smo3/data.tar.gz


In the following code, we setup the root directory for our project, extract the dataset to the root directory, and download the label files.

In [ ]:
import os

# Let's make our assignment directory
CS163_path = './CS163_W24'
os.makedirs(os.path.join(CS163_path, 'Assignment2', 'data'), exist_ok=True)
# os.makedirs will create directories recursively, i.e., it will create the
# directories and any missing parent directories if they do not exist.

# Now, let's specify the assignment path we will be working with as the root.
root_dir = os.path.join(CS163_path, 'Assignment2')

import os
import tarfile
import urllib.request

from tqdm import tqdm


def setup(file_link_dict={}):
    # Open the tar.gz file
    tar = tarfile.open("data.tar.gz", "r:gz")
    # Extract the file "./Assignment2/data" folder
    total_size = sum(f.size for f in tar.getmembers())
    with tqdm(total=total_size,
              unit="B",
              unit_scale=True,
              desc="Extracting tar.gz file") as pbar:
        for member in tar.getmembers():
            tar.extract(member, os.path.join(root_dir, 'data'))
            pbar.update(member.size)
    # Close the tar.gz file
    tar.close()

    # Next, we download the train/val/test txt files:
    for file_name, file_link in file_link_dict.items():
        print(f'Downloding {file_name}.txt from {file_link}')
        urllib.request.urlretrieve(file_link,
                                   f'{root_dir}/data/{file_name}.txt')


val_url = 'https://raw.githubusercontent.com/CSAILVision/miniplaces/master/data/val.txt'
train_url = 'https://raw.githubusercontent.com/CSAILVision/miniplaces/master/data/train.txt'
setup(file_link_dict={'train': train_url, 'val': val_url}, )

Also, it is a good habit to setup the seed before running any experiment to ensure reproducibility.

In [ ]:
# Also, seed everything for reproducibility
# code from https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964#file-seed_everything-py
def seed_everything(seed: int):
    import os
    import random

    import numpy as np
    import torch
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(0)

In the following, fill in the PyTorch MiniPlaces dataset you implemented in Assignment 1(we will release the key for A1 after it's due.)
Since, we will be focused on convolutional neural networks for this assignment, **there's no need to flatten the image to a vector**. In addition, to fully utilize the modelling power of convolutional neural networks, we will not downsample the image and use its original size. Therefore, the output size of the image tensor should be $[3, 128, 128]$, please update the image transformation in the codeblock below

In [ ]:
import torch
from PIL import Image
from torchvision import transforms

image_net_mean = torch.Tensor([0.485, 0.456, 0.406])
image_net_std = torch.Tensor([0.229, 0.224, 0.225])
# Define data transformation
data_transform = transforms.Compose([
    ################# Your Implementations #####################################
    # TODO: Resize image to 128x128,
    ################# End of your Implementations ##############################
    transforms.ToTensor(),

    ################# Your Implementations #####################################
    # TODO: Normalize image using ImageNet statistics
    ################# End of your Implementations ##############################
])

Also, we will be using the same Pytorch dataset definition in A1, just copy your implementations here:

In [ ]:
import os

from torch.utils.data import DataLoader, Dataset


class MiniPlaces(Dataset):

    def __init__(self, root_dir, split, transform=None, label_dict=None):
        """
        Initialize the MiniPlaces dataset with the root directory for the images,
        the split (train/val/test), an optional data transformation,
        and an optional label dictionary.

        Args:
            root_dir (str): Root directory for the MiniPlaces images.
            split (str): Split to use ('train', 'val', or 'test').
            transform (callable, optional): Optional data transformation to apply to the images.
            label_dict (dict, optional): Optional dictionary mapping integer labels to class names.
        """
        assert split in ['train', 'val', 'test']
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.filenames = []
        self.labels = []

        # Take a second to think why we need this line.
        # Hints: training set / validation set / test set.
        self.label_dict = label_dict if label_dict is not None else {}

        # You should
        #   1. Load the train/val/test text file based on the `split` argument and
        #     store the image filenames and labels.
        #   2. Extract the class names from the image filenames and store them in
        #     self.label_dict.
        #   3. Construct a label dict that maps integer labels to class names, if
        #     the current split is "train"
        ################# Your Implementations #################################

        ################# End of your Implementations ##########################

    def __len__(self):
        """
        Return the number of images in the dataset.

        Returns:
            int: Number of images in the dataset.
        """
        dataset_len = 0
        ################# Your Implementations #################################
        # Return the number of images in the dataset

        ################# End of your Implementations ##########################
        return dataset_len

    def __getitem__(self, idx):
        """
        Return a single image and its corresponding label when given an index.

        Args:
            idx (int): Index of the image to retrieve.

        Returns:
            tuple: Tuple containing the image and its label.
        """
        ################# Your Implementations #################################
        # Load and preprocess image using self.root_dir,
        # self.filenames[idx], and self.transform (if specified)

        ################# End of your Implementations ##########################
        return image, label


data_root = os.path.join(root_dir, 'data')
# Create MiniPlaces dataset object
miniplaces_train = MiniPlaces(data_root,
                              split='train',
                              transform=data_transform)

miniplaces_val = MiniPlaces(data_root,
                            split='val',
                            transform=data_transform,
                            label_dict=miniplaces_train.label_dict)

# Create the dataloaders
# Define the batch size and number of workers
batch_size = 64
num_workers = 2

# Create DataLoader for training and validation sets
train_loader = DataLoader(miniplaces_train,
                          batch_size=batch_size,
                          num_workers=num_workers,
                          shuffle=True)
val_loader = DataLoader(miniplaces_val,
                        batch_size=batch_size,
                        num_workers=num_workers,
                        shuffle=False)


Then, define the training and evaluation functions to train and test the CNN (Same as Assignment 1, You don't need to modify this part):

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, device,
          num_epochs):
    """
    Train the MLP classifier on the training set and evaluate it on the validation set every epoch.

    Args:
        model (MLP): MLP classifier to train.
        train_loader (torch.utils.data.DataLoader): Data loader for the training set.
        val_loader (torch.utils.data.DataLoader): Data loader for the validation set.
        optimizer (torch.optim.Optimizer): Optimizer to use for training.
        criterion (callable): Loss function to use for training.
        device (torch.device): Device to use for training.
        num_epochs (int): Number of epochs to train the model.
    """
    # Place model on device
    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        # Use tqdm to display a progress bar during training
        with tqdm(total=len(train_loader),
                  desc=f'Epoch {epoch + 1}/{num_epochs}',
                  position=0,
                  leave=True) as pbar:
            for inputs, labels in train_loader:
                # Move inputs and labels to device
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero out gradients
                optimizer.zero_grad()

                # Compute the logits and loss
                logits = model(inputs)
                loss = criterion(logits, labels)

                # Backpropagate the loss
                loss.backward()

                # Update the weights
                optimizer.step()

                # Update the progress bar
                pbar.update(1)
                pbar.set_postfix(loss=loss.item())

        # Evaluate the model on the validation set
        avg_loss, accuracy = evaluate(model, val_loader, criterion, device)
        print(
            f'Validation set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}'
        )


def evaluate(model, test_loader, criterion, device):
    """
    Evaluate the MLP classifier on the test set.

    Args:
        model (MLP): MLP classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.

    Returns:
        float: Average loss on the test set.
        float: Accuracy on the test set.
    """
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        total_loss = 0.0
        num_correct = 0
        num_samples = 0

        for inputs, labels in test_loader:
            # Move inputs and labels to device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Compute the logits and loss
            logits = model(inputs)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            # Compute the accuracy
            _, predictions = torch.max(logits, dim=1)
            num_correct += (predictions == labels).sum().item()
            num_samples += len(inputs)

    # Compute the average loss and accuracy
    avg_loss = total_loss / len(test_loader)
    accuracy = num_correct / num_samples

    return avg_loss, accuracy

Let's setting up the device, Assignment 2 involves much more computaton than Assignment 1, and hence using a GPU is preferred

In [ ]:
import torch
import torch.nn.functional as F

# Define the device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

In [ ]:
# This line of code gives you the information about GPU
! nvidia-smi

## Q1: Convolutional Neural Network (CNN) (30 pts)

In this question, you will learn how to implement convolutional neural networks (CNNs) using PyTorch.

Similarly as Q2, we will first build a two-layer convolutional neural networks using PyTorch built-in function `nn.Conv2D` and then build the convolution operation from scratch.

### Q1.1 Fast Conv (10 pts)

Fill in the code below to implement a simple CNN similar to the MLP we built in A1, we can use built-in PyTorch Operations (e.g `nn.conv2d`) to build the network.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class FastConv(nn.Module):

    def __init__(self,
                 input_channels,
                 conv_hidden_channels,
                 conv_out_channels,
                 input_size=(64, 64),
                 dropout_rate1=0.25,
                 dropout_rate2=0.5,
                 fc_out_channels=128,
                 num_classes=100,
                 kernel_size=3,
                 stride=2,
                 padding=1):
        """
        Args:
          input_channels (int): Number of channels in the input image.
          conv_hidden_channels (int): Number of channels in the first convolutional layer.
          conv_out_channels (int): Number of channels in the second convolutional layer.
          input_size (tuple, optional): Height and width of the input image. (default: (64,64))
          dropout_rate1, dropout_rate2 (float, optional): Dropout rate for
              the first/second dropout layer. (default: 0.25, 0.5)
          fc_out_channels (int, optional): Number of neurons in the first fully
              connected layer. (default: 128)
          num_classes (int, optional): Number of classes in the final output layer. (default: 100)
          kernel_size, stride, padding (int, optional): Parameters of convolutional layers.

        Initialize a convolutional neural network.
        You can use Pytorch's built-in functions like, nn.Conv2d, nn.MaxPool2d, etc.
        Input and output shapes of each layer:
        1) conv1: (batch_size, input_channels, H, W) -> (batch_size, conv_hidden_channels, H//2, W//2)
        2) conv2: (batch_size, conv_hidden_channels, H//2, W//2) -> (batch_size, conv_out_channels, H//4, W//4)
        3) max_pooling: (batch_size, conv_hidden_channels, H//4, W//4) -> (batch_size, conv_out_channels, H//8, W//8)
        4) fc1: (batch_size, flatten_size) -> (batch_size, fc_out_channels)
        5) fc2: (batch_size, fc_out_channels) -> (batch_size, num_classes)
        """
        super().__init__()

        self.conv1 = None
        self.conv2 = None
        self.max_pooling = None
        self.dropout1 = None
        self.dropout2 = None
        self.fc1 = None
        self.fc2 = None
        ################# Your Implementations #################################
        # TODO: Define the layers of the convolutional neural network
        # replace "None"s with your implementations.
        # All you need to do is to pass the input arguments to different constructors

        ################# End of your Implementations ##########################

    def forward(self, x, return_intermediate=False):
        """
        Forward pass of the convolutional neural network.
        The input tensor 'x' should pass through the following layers:
        1) conv1: (batch_size, input_channels, H, W) -> (batch_size, conv_hidden_channels, H//2, W//2)
        2) Apply relu.
        3) conv2: (batch_size, conv_hidden_channels, H//2, W//2) -> (batch_size, conv_out_channels, H//4, W//4)
        4) max_pooling: Perform 2*2 max pooling with stride 2 on the output from conv2
        5) dropout1: Perform dropout2d on the output from max_pooling
        6) Flatten the output from dropout1
        7) fc1: Pass through a fully connected layer
        8) dropout2: Perform dropout on the output from fc1
        9) Apply relu.
        7) fc2: Pass the output from the actiction layer to through a fully connected
                layer to produce the final output
        """
        ################# Your Implementations #################################
        # TODO: Implement the forward pass of the convolutional neural network

        ################# End of your Implementations ##########################
        return x

In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
model = FastConv(input_channels=3,
                 conv_hidden_channels=64,
                 conv_out_channels=128,
                 input_size=(128, 128),
                 dropout_rate1=0.25,
                 dropout_rate2=0.5,
                 fc_out_channels=128,
                 kernel_size=3,
                 stride=2,
                 padding=1,
                 num_classes=len(miniplaces_train.label_dict))

# Let's use the built-in optimizer for a full version of SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# For loss function, your implementation and the built-in loss function should
# be almost identical.
criterion = nn.CrossEntropyLoss()

# Train the model
train(model,
      train_loader,
      val_loader,
      optimizer,
      criterion,
      device,
      num_epochs=5)

You should get around 14.93% accuracy after the first epoch and 18.27% accuracy after five epochs, which is a huge improvement from using MLPs.

### Q1.2 Building 2D Convolutional layer from sratch (20pts)

Next, we will build convolutional neural networks from scratch.


Let's ignore channels for now and see how this works with two-dimensional data had hidden representations. In the following figure, the input is a two-dimensional tensor with hegith of 3 and width of 3. We mark the shape of the tensor as $3\times3$. The height and width of the kernel are both 2. The shape of the kernel window (or convolution window) is given by the height and width of the kernel (here it is $2\times2$).

![correlation](https://web.cs.ucla.edu/~smo3/cs188/assignment2/correlation.png)


In the two-dimensional convolution operation, we begin with the convolution window position at the upper-left corner of the input tensor and slide it across the intput tensor, both from left to right and top to bottom. When the convolution window slides to a certain position, the input subtensor contained in that window and the kernel tensor are multiplied elementwise and the resulting tensor is summed up yielding a single scalar value. This result gives the value of the output tensor at the corresponding location. Here, the output tensor has a height of 2 and width of 2 and the four elements are derived from the two-dimensional cross-correlation operation:
$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.
$$

Note that along each axis, the output size
is slightly smaller than the input size.
Because the kernel has width and height greater than one,
we can only properly compute the cross-correlation
for locations where the kernel fits wholly within the image,
the output size is given by the input size $n_h \times n_w$
minus the size of the convolution kernel $k_h \times k_w$
via

$$(n_h-k_h+1) \times (n_w-k_w+1).$$

This is the case since we need enough space
to "shift" the convolution kernel across the image.

#### Q1.2.1 One Channel Convolution (5pts)
That's start with 2D convolution with only 1 input channel and 1 kernel.
If your implementation is right, you'll get:
```
tensor([[19., 25.],
        [37., 43.]])
```

In [ ]:
def conv2d(X, K):
    """
    Compute 2D convolution. You are supposed to use matrix multiplication
    to achieve the 2D convolution instead of torch build in convolution method.

        Args:
          X (tensor): Input tensor in the shape of (n_h,n_w).
          K (tensor): Kernel tensor in the shape of (k_h,k_w).

        Return:
          Y (tensor): Output tensor in the shape of (H,W).

    """
    Y = None
    ################# Your Implementations #####################################
    # Hint: Using two for loops

    ################# End of your Implementations ##############################
    return Y

In [ ]:
# Try to debug using this code segment:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
conv2d(X, K)

#### Q1.2.2  One Channel Convolution with Padding and Stride (5pts)

Since we typically use small kernels,
for any given convolution,
we might only lose a few pixels,
but this can add up as we apply
many successive convolutional layers.
One straightforward solution to this problem
is to add extra pixels of filler around the boundary of our input image,
thus increasing the effective size of the image.
Typically, we set the values of the extra pixels to zero.
In the following image, we pad a $3 \times 3$ input,
increasing its size to $5 \times 5$.
The corresponding output then increases to a $4 \times 4$ matrix.
The shaded portions are the first output element as well as the input and kernel tensor elements used for the output computation: $0\times0+0\times1+0\times2+0\times3=0$.

![Two-dimensional cross-correlation with padding.](https://web.cs.ucla.edu/~smo3/cs188/assignment2/conv-pad.png)

In general, if we add a total of $p_h$ rows of padding
(roughly half on top and half on bottom)
and a total of $p_w$ columns of padding
(roughly half on the left and half on the right),
the output shape will be

$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1).$$

This means that the height and width of the output
will increase by $p_h$ and $p_w$, respectively.

In many cases, we will want to set $p_h=k_h-1$ and $p_w=k_w-1$
to give the input and output the same height and width.
This will make it easier to predict the output shape of each layer
when constructing the network.

When computing the cross-correlation,
we start with the convolution window
at the upper-left corner of the input tensor,
and then slide it over all locations both down and to the right.
In the previous examples, we defaulted to sliding one element at a time.
However, sometimes, either for computational efficiency
or because we wish to downsample,
we move our window more than one element at a time,
skipping the intermediate locations. This is particularly useful if the convolution
kernel is large since it captures a large area of the underlying image.

We refer to the number of rows and columns traversed per slide as *stride*.
So far, we have used strides of 1, both for height and width.
Sometimes, we may want to use a larger stride.
The following image shows a two-dimensional cross-correlation operation
with a stride of 3 vertically and 2 horizontally.
The shaded portions are the output elements as well as the input and kernel tensor elements used for the output computation: $0\times0+0\times1+1\times2+2\times3=8$, $0\times0+6\times1+0\times2+0\times3=6$.
We can see that when the second element of the first column is generated,
the convolution window slides down three rows.
The convolution window slides two columns to the right
when the second element of the first row is generated.
When the convolution window continues to slide two columns to the right on the input,
there is no output because the input element cannot fill the window
(unless we add another column of padding).

![Cross-correlation with strides of 3 and 2 for height and width, respectively.](https://web.cs.ucla.edu/~smo3/cs188/assignment2/conv-stride.png)

In general, when the stride for the height is $s_h$
and the stride for the width is $s_w$, the output shape is

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$



In the next cell, you will need to add padding and stride to the conv2d function.

If your implementation is right, the correct output is
```
tensor([[19., 31.],
        [50., 62.]])
```

In [ ]:
def conv2d_padding_stride(X, K, padding=0, stride=0):
    """Compute 2D convolution with stride. Here the stride applied both
       horizontally and vertically.

        Args:
          X (tensor): Input tensor in the shape of (n_h,n_w).
          K (tensor): Kernel tensor in the shape of (k_h,k_w).
          padding (int): padding in both dimensions (p_h, p_w).
          stride (int): Stride in both dimensions (s_h, s_w).

        Return:
          Y (tensor): Output tensor in the shape of (H,W)
    """
    Y = None
    ################# Your Implementations #####################################

    ################# End of your Implementations ##############################
    return Y

In [ ]:
# Try to debug using this code segment:
X = torch.tensor([[0.0, 1.0, 2.0, 3.0], [3.0, 4.0, 5.0, 6.0],
                  [6.0, 7.0, 8.0, 9.0], [8.0, 9.0, 10.0, 11.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
conv2d_padding_stride(X, K, padding=0, stride=2)

#### Q1.2.3 Multiple Channels 2D Convlution (5pts)
When the input data contains multiple channels,
we need to construct a convolution kernel
with the same number of input channels as the input data,
so that it can perform cross-correlation with the input data.
Assuming that the number of channels for the input data is $c_i$,
the number of input channels of the convolution kernel also needs to be $c_i$. If our convolution kernel's window shape is $k_h\times k_w$,
then when $c_i=1$, we can think of our convolution kernel
as just a two-dimensional tensor of shape $k_h\times k_w$.

However, when $c_i>1$, we need a kernel
that contains a tensor of shape $k_h\times k_w$ for *every* input channel. Concatenating these $c_i$ tensors together
yields a convolution kernel of shape $c_i\times k_h\times k_w$.
Since the input and convolution kernel each have $c_i$ channels,
we can perform a cross-correlation operation
on the two-dimensional tensor of the input
and the two-dimensional tensor of the convolution kernel
for each channel, adding the $c_i$ results together
(summing over the channels)
to yield a two-dimensional tensor.
This is the result of a two-dimensional cross-correlation
between a multi-channel input and
a multi-input-channel convolution kernel.

The following image provides an example
of a two-dimensional cross-correlation with two input channels.
The shaded portions are the first output element
as well as the input and kernel tensor elements used for the output computation:
$(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1+3\times2+4\times3)=56$.

![Cross-correlation computation with 2 input channels.](https://web.cs.ucla.edu/~smo3/cs188/assignment2/conv-multi-in.png)


To make sure we really understand what is going on here,
we can (**implement cross-correlation operations with multiple input channels**) ourselves.
Notice that all we are doing is performing a cross-correlation operation
per channel and then adding up the results.

Then, we'll consider the cases where there are multiple input channels. If your implementation is right, the correct output is


```
tensor([[  4.,  26.,  18.],
        [ 57., 120.,  55.],
        [ 29.,  46.,  13.]])
```



In [ ]:
def conv2d_multi_in(X, K, padding=0, stride=0) -> torch.tensor:
    """Compute 2D cross-correlation with multiple input channels.
        Here, X and K should have the same chanel number

        Args:
          X (tensor): Input tensor in the shape of (in_Channels,n_h,n_w).
          K (tensor): Kernel tensor (in_Channels,k_h,k_w)
          padding (int): padding in H,W dimensions (p_h, p_w).
          stride (int): Stride in H,W dimensions (s_h, s_w).

        Return:
          Y (tensor): Output tensor in the shape of (H,W)

    """
    # Hint： Iterate through the 0th dimension (channel) of K first, then stack them up, then take the sum
    Y = None
    ################# Your Implementations #####################################

    ################# End of your Implementations ##############################
    return Y

In [ ]:
# Try to debug using this code segment:
X = torch.tensor([[[0.0, 1.0, 2.0, 3.0], [3.0, 4.0, 5.0, 6.0],
                   [6.0, 7.0, 8.0, 9.0], [9.0, 10.0, 11.0, 12.0]],
                  [[1.0, 2.0, 3.0, 4.0], [4.0, 5.0, 6.0, 7.0],
                   [7.0, 8.0, 9.0, 10.0], [10.0, 11.0, 12.0, 13.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

out = conv2d_multi_in(X, K, padding=1, stride=2)
out

In [ ]:
# Compare our method with torch build-in method
X = (torch.rand(2, 10, 10)).to(torch.float32)
K = (torch.rand(2, 3, 3)).to(torch.float32)
for i in [1, 2, 3, 4, 5]:
    for j in [1, 2, 3, 4, 5]:
        out = conv2d_multi_in(X, K, padding=i, stride=j)
        answer = F.conv2d(X.unsqueeze(0), K.unsqueeze(0), stride=j,
                          padding=i).squeeze(0).squeeze(0)
        equal = torch.allclose(out, answer)

        if equal:
            print('Good! For padding: %d and stride: %d, the output match' %
                  (i, j))
        else:
            print('Uh-oh! The output are different')
            break

Then, we'll consider the cases where there are multiple input channels and multiple output channels. We will be also adding a bias tensor $B$ on each output channel. Notice the input tensor X has a batch demension.

In [ ]:
def batched_conv2d_multi_in_out(X, K, B, padding=0, stride=0) -> torch.tensor:
    """Compute 2D cross-correlation with multiple input channels and multiple
        output channels.

        Args:
          X (tensor): Input tensor in the shape of (batch_size,in_channels,n_h,n_w).
          K (tensor): Kernel tensor (out_channels,in_channels,k_h,k_w).
          B (tensor): Bias tensor (out_channels)
          padding (int): padding in H,W dimensions (p_h, p_w).
          stride (int): Stride in H,W dimensions (s_h, s_w).

        Return:
          Y (tensor): Output tensor in the shape of (batch_size,out_channels,H,W)

    """
    # Hint: Loop though the batch dimension and then loop though the out_channel
    #       dimension
    Y = None
    ################# Your Implementations #####################################

    ################# End of your Implementations ##############################
    return Y

In [ ]:
# Compare our method with torch build-in method
X = (torch.rand(2, 2, 10, 10)).to(torch.float32)
K = (torch.rand(2, 2, 3, 3)).to(torch.float32)
B = (torch.rand(2)).to(torch.float32)
for i in [1, 2, 3, 4, 5]:
    for j in [1, 2, 3, 4, 5]:
        out = batched_conv2d_multi_in_out(X, K, B, padding=i, stride=j)
        answer = F.conv2d(X, K, B, stride=j, padding=i)
        equal = torch.allclose(out, answer)

        if equal:
            print('Good! For padding: %d and stride: %d, the output match' %
                  (i, j))
        else:
            print('Uh-oh! The output are different')
            break

#### Q1.2.4 Try your own implementation (5pts)

In the next task, you will implement your own 2D convolution neural network with the custom convolution operator you just wrote.

This task should be relatively straightforward. Just to pay attention to shape of the weights and biases of the convolution.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CustomConv2d(torch.nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 padding=0):
        super(CustomConv2d, self).__init__()
        # Initialize weights using random initialization with 0.01 std
        # Initialize bias using zero initialization
        # Save the stride and padding info
        ################# Your Implementations #################################

        ################# End of your Implementations ##########################

    def forward(self, x):
        # call your implemented function batched_conv2d_multi_in_out/or F.conv2d here
        ################# Your Implementations #################################
      
        ################# End of your Implementations ##########################


class CustomConv(nn.Module):

    def __init__(self,
                 input_channels,
                 conv_hidden_channels,
                 conv_out_channels,
                 input_size=(64, 64),
                 dropout_rate1=0.25,
                 dropout_rate2=0.5,
                 fc_out_channels=128,
                 num_classes=100,
                 kernel_size=3,
                 stride=2,
                 padding=1):
        """
        Args:
          input_channels (int): Number of channels in the input image.
          conv_hidden_channels (int): Number of channels in the first convolutional layer.
          conv_out_channels (int): Number of channels in the second convolutional layer.
          input_size (tuple, optional): Height and width of the input image. (default: (64,64))
          dropout_rate1, dropout_rate2 (float, optional): Dropout rate for
              the first/second dropout layer. (default: 0.25, 0.5)
          fc_out_channels (int, optional): Number of neurons in the first fully
              connected layer. (default: 128)
          num_classes (int, optional): Number of classes in the final output layer. (default: 100)
          kernel_size, stride, padding (int, optional): Parameters of convolutional layers.

        Initialize a convolutional neural network.
        You should use your own convolution operator CustomConv2d.
        Input and output shapes of each layer:
        1) conv1: (batch_size, input_channels, H, W) -> (batch_size, conv_hidden_channels, H//2, W//2)
        2) conv2: (batch_size, conv_hidden_channels, H//2, W//2) -> (batch_size, conv_out_channels, H//4, W//4)
        3) max_pooling: (batch_size, conv_out_channels, H//4, W//4) -> (batch_size, conv_out_channels, H//8, W//8)
        4) fc1: (batch_size, flatten_size) -> (batch_size, fc_out_channels)
        5) fc2: (batch_size, fc_out_channels) -> (batch_size, num_classes)
        """
        super().__init__()

        self.conv1 = None
        self.conv2 = None
        self.max_pooling = None
        self.dropout1 = None
        self.dropout2 = None
        self.fc1 = None
        self.fc2 = None
        ################# Your Implementations #################################
        # TODO: Define the layers of the convolutional neural network
        # All you need to do is to replace your previous implementatin with CustomConv2d
        

        ################# End of your Implementations ##########################

    def forward(self, x, return_intermediate=False):
        """
        Forward pass of the convolutional neural network.
        The input tensor 'x' should pass through the following layers:
        1) conv1: (batch_size, input_channels, H, W) -> (batch_size, conv_hidden_channels, H//2, W//2)
        2) Apply relu.
        3) conv2: (batch_size, conv_hidden_channels, H//2, W//2) -> (batch_size, conv_out_channels, H//4, W//4)
        4) max_pooling: Perform 2*2 max pooling with stride 2 on the output from conv2
        5) dropout1: Perform dropout2d on the output from max_pooling
        6) Flatten the output from dropout1
        7) fc1: Pass through a fully connected layer
        8) dropout2: Perform dropout on the output from fc1
        9) Apply relu.
        7) fc2: Pass the output from the actiction layer to through a fully connected
                layer to produce the final output
        """
        ################# Your Implementations #################################
        # TODO: Implement the forward pass of the convolutional neural network
        

        ################# End of your Implementations ##########################
        return x

You may notice that using our own implemented `batched_conv2d_multi_in_out` convolution operator is pretty slow, as it uses forloop to comupte the convolution and does not utilize the parallel computation power of a GPU. In that case, we can directly use the gpu-accelerated built-in operator `F.conv2d` after we have verified it outputs the same results as `batched_conv2d_multi_in_out`.

In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
model = CustomConv(input_channels=3,
                   conv_hidden_channels=64,
                   conv_out_channels=128,
                   input_size=(128, 128),
                   dropout_rate1=0.25,
                   dropout_rate2=0.5,
                   fc_out_channels=128,
                   kernel_size=3,
                   stride=2,
                   padding=1,
                   num_classes=len(miniplaces_train.label_dict))

# Let's use the built-in optimizer for a full version of SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# For loss function, your implementation and the built-in loss function should
# be almost identical.
criterion = nn.CrossEntropyLoss()

# Train the model
train(model,
      train_loader,
      val_loader,
      optimizer,
      criterion,
      device,
      num_epochs=5)

(0 pts) You may notice the accuracy of our own implmentation is lower than using the built-in modules. Think about why (hint: initialization)?

## Q2 ResNet (50 pts)

Now you haved gained some basic experience with CNNs. In this question, you will learn how to build ResNet and how to use pretrained model and apply transfer learning on ResNet18.

### Q2.1 Build ResNet from Scratch (10 pts)

<p align="center">
    <img src="https://drive.google.com/uc?id=1qmjTRqYAUZfJnjbA_X3sEz9FpISQh94k" width="800" />
</p>

In this question, we will build ResNet-18 from Scratch.  First, let's define the basic residual blocks. Each basic residual block contains two 3-by-3 convolution, followed by batch normalization and relu activation. The final output is added throught the residual connection to the original input.

In [ ]:
class ResidualBlock(nn.Module):

    def __init__(self, in_channels, out_channels, downsample=False):
        '''
        Initialize a basic residual block.
        Depending on whether downsample is True, there are two types of basic blocks in Resnet 18,
        one to downsample the input and the other maintains the same size.
        You can use Pytorch's functions.

        Input and output shapes of each layer:
        1) conv1 (3*3 kernel, no bias): (batch_size, in_channels, H, W) -> (batch_size, out_channels, H, W) if downsample=False
           conv1 (3*3 kernel, no bias): (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) if downsample=True, with stride of 2
        2) conv2 (3*3 kernel, no bias): (batch_size, out_channels, H, W) -> (batch_size, out_channels, H, W)
        3) conv3 (optional) if downsample=True (1*1 kernel, no bias): (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) with stride of 2
        '''
        super(ResidualBlock, self).__init__()
        ################# Your Implementations #################################

        ################# End of your Implementations ##########################

    def forward(self, x):
        """
        Forward pass of the basic residual block.
        The input tensor 'x' should pass through the following layers:
        1) conv1: (batch_size, in_channels, H, W) -> (batch_size, out_channels, H, W)
           conv1: (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) if downsample=True
        2) Apply batch normalization after conv1.
        3) Apply relu activation.
        4) conv2: (batch_size, out_channels, H, W) -> (batch_size, out_channels, H, W)
        5) Apply batch normalization after conv2.
        6) (Optional) if downsample=True, conv3: (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) on the original input
        7) (Optional) if downsample=True, apply batch normalization after conv3.
        8) Add the residual value to the original input
        9) Apply relu activation in the end.
        """
        ################# Your Implementations #################################
        # TODO: Implement the forward pass of the basic residual block.

        ################# End of your Implementations ##########################
        return out

Besides the basic residual blocks, ResNet also groups two residual blocks as a single residual layer or stage, and there are four stages for resnet-18, implement the four layers in the code below following the given instructions.

In [ ]:
class ResNet18(nn.Module):

    def __init__(self, num_classes=100):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        # Initial convolution
        self.conv1 = nn.Conv2d(3,
                               64,
                               kernel_size=7,
                               stride=2,
                               padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # define four residual sequential layers, each layer consists of two residual blocks
        # if the layer has the same input and output channels, do not apply downsampling
        # if the layer has doubled output channels from the input, apply downsample on the first residual blocks with also doubled output channels
        # the second residual block would always have the same input and output channels without downsample
        # layer1: input channel: 64, output channel: 64
        # layer2: input channel: 64, output channel: 128
        # layer3: input channel: 128, output channel: 256
        # layer4: input channel: 256, output channel: 512
        # use nn.Sequential to concatenate the two blocks
        ################# Your Implementations #################################

        ################# End of your Implementations ##########################
        # Average Pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # Fully connected layer
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


# Create a ResNet-18 model
model = ResNet18(num_classes=100)

Let's see how ResNet-18 performs on the TinyPlaces image classification benchmark:

In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
model = ResNet18(num_classes=len(miniplaces_train.label_dict))

# Let's use the built-in optimizer for a full version of SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# For loss function, your implementation and the built-in loss function should
# be almost identical.
criterion = nn.CrossEntropyLoss()

# Train the model
train(model,
      train_loader,
      val_loader,
      optimizer,
      criterion,
      device,
      num_epochs=5)

You should be getting around 20.42% accuracy after the first epoch and 35.57% accuracy after five epochs.

### Q2.2 Finetuning ResNet from PyTorch Pretrained Model (15 pts)

#### Q2.2.1 Loading Pretrained ResNet from PyTorch (5 pts)

Instead of building ResNet-18 from scatch, PyTorch also provides pre-built ResNet models from `torchvision.models` and the corresponding pretrained checkpoints on ImageNet to facilitate computer vision study and research. Fill in the following code to load a pretrained ResNet model from PyTorch with different finetuning modes.

In [ ]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
from torch import nn


class Resnet(nn.Module):

    def __init__(self, mode='finetune', num_classes=100):
        super().__init__()
        """
        use the resnet18 model from torchvision models. Remember to set pretrained as true

        mode has two options:
        1) linear: For this model, we want to freeze resnet18 features, then train a linear
            classifier which takes the features before resnet.fc (we do not want
            the pretrained resnet.fc for 1000 classes). And then write our own FC layer: which takes in the features and
            output scores of size 100 (because we have 100 categories).
            Because we want to freeze resnet18 features, we have to iterate through parameters()
            of our model, and manually set some parameters to requires_grad = False
            Or use other methods to freeze the features
        2) finetune: Same as 1), except that we we do not need to freeze the features and
           can finetune on the pretrained resnet model.
        """
        ################# Your Implementations #################################

        ################# End of your Implementations ##########################

    def forward(self, x):
        ################# Your Implementations #################################

        ################# End of your Implementations ##########################
        return x

    def to(self, device):
        return self.resnet.to(device=device)

#### Q2.2.2 Pretrained ResNet features + Linear Classifier

Then we use the “linear” mode in Resnet class to only train a linear classifier based on extracted ResNet features. Remember to freeze the features of ResNet. In this implementation we use linear classifier to do classification on the dataset.

In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
resnet_linear = Resnet(mode='linear',
                       num_classes=len(miniplaces_train.label_dict))

optimizer = torch.optim.SGD(resnet_linear.parameters(), lr=0.01, momentum=0.9)

criterion = nn.CrossEntropyLoss()

# Train the model
train(resnet_linear,
      train_loader,
      val_loader,
      optimizer,
      criterion,
      device,
      num_epochs=5)

I'm getting 39.36% accuracy after the first epoch and 39.99% accuracy after five epochs.

#### Q2.2.3 Fully finetune with same learning rate
Fine-tuning a pre-trained ResNet model on a specific task, such as image classification or object detection, can help improve its performance on that task by adapting the model's learned representations to the particular dataset and task at hand. This is because the pre-trained ResNet has already learned useful feature representations on a large dataset, and fine-tuning allows it to further specialize those representations to the specific task.

In this implementation, instead of freezing ResNet features, we want to finetune it. Here we use the "finetune" mode in Resnet class, the implementation is the same as the "linear" mode, except that we do not need to set requires_grad to False to freeze the features. Here, we are going to finetune the ResNet end-to-end finetune with SGD optimizer. Setting lr to 0.01 and momentum to 0.9.

In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
resnet_finetune1 = Resnet(mode='finetune',
                          num_classes=len(miniplaces_train.label_dict))

optimizer = torch.optim.SGD(resnet_finetune1.parameters(),
                            lr=0.01,
                            momentum=0.9)

criterion = nn.CrossEntropyLoss()

train(resnet_finetune1,
      train_loader,
      val_loader,
      optimizer,
      criterion,
      device,
      num_epochs=5)

I'm getting 45.16% accuracy after the first epoch and 45.85% accuracy after five epochs.

#### Q2.2.4 Fully finetune with different learning rates (5 pts)

Lower layers in a network typically learn low-level features such as edges and textures that are useful across a wide range of tasks, and therefore may require smaller updates to prevent overfitting. Meanwhile, higher layers may learn more task-specific features that require larger updates to improve performance.

Here, we finetune the pretrained ResNet with SGD optimizer with momentum = 0.9. We will need to assign different learning rate to different layers.


*   Setting lr to 0.01 for the last fc layer.
*   Setting lr to 0.001 for the rest layers.




In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
resnet_finetune2 = Resnet(mode='finetune',
                          num_classes=len(miniplaces_train.label_dict))

optimizer = None
################# Your Implementations #########################################
# Hint, read the official document for SGD to check how to set different group of params
# https://pytorch.org/docs/stable/generated/torch.optim.SGD.html

# Create a list of parameter groups with different learning rates

################# End of your Implementations ##################################

criterion = nn.CrossEntropyLoss()

# Train the model
train(resnet_finetune2,
      train_loader,
      val_loader,
      optimizer,
      criterion,
      device,
      num_epochs=5)

I'm getting 49.49% accuracy after the first epoch and 50.49% accuracy after five epochs.

#### Q2.2.5 Finetune with a few frozen layers (5 pts)
Freezing some layers during fine-tuning can help to prevent overfitting and speed up the training process. When we fine-tune a pre-trained neural network, we typically want to retain the learned feature representations in the lower layers of the network, which are often more general and transferable across different tasks. By freezing these lower layers, we prevent their weights from being updated during fine-tuning, which helps to ensure that the model retains its learned feature representations. This can be particularly important if we have a small amount of data available for the specific task we are fine-tuning for.

In this step, you will define a new optimizer to forzen all the parameters in resnet.layer1 and resnet.layer2.

Here, we finetune the pretrained ResNet with SGD optimizer with momentum = 0.9. We will need to assign different learning rate to different layers.


*   Setting lr to 0.01 for the last fc layer.
*   Setting lr to 0 for the resnet.layer1 and resnet.layer2 layers.
*   Setting lr to 0.001 for the rest layers.

In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
resnet_finetune3 = Resnet(mode='finetune',
                          num_classes=len(miniplaces_train.label_dict))

optimizer = None
################# Your Implementations #########################################

################# End of your Implementations ##################################

criterion = nn.CrossEntropyLoss()

# Train the model
train(resnet_finetune3,
      train_loader,
      val_loader,
      optimizer,
      criterion,
      device,
      num_epochs=5)

I'm getting 49.18% accuracy after the first epoch and 49.87% accuracy after five epochs.

### Q2.3 Visualization of ResNet (10 pts)
The attention maps in Convolutional Neural Networks (CNNs) are used to visualize the regions of an input image that the model is paying attention to when making predictions. These attention maps convey strong messages about which features the CNNs are looking at.  

<p align="center">
    <img src="https://drive.google.com/uc?id=1HGv3yE-UaCIw4AnyKu4GUolapU7XV9Cd" width="800" />
</p>


Now we will implement Prof. Zhou's famous paper: "Learning Deep Features for Discriminative Localization", in which he proposed class activation mapping (CAM). The main equation is:
$$S_{c}=\sum_{k} w_{k}^{c} \sum_{x, y} f_{k}(x, y)=\sum_{x, y} \sum_{k} w_{k}^{c} f_{k}(x, y)$$
where $f_{k}(x, y)$ represents the activation of unit k in the last convolutional layer, which is layer4 in resnet18.

For more detailed implementation, please refer to the github repo: https://github.com/zhoubolei/CAM

Please implement the CAM function. Specifically, given convolutional features, weights and a class index, CAM will output the reasons for classifying the image to the class, thus making CNN interpretable.

In [ ]:
def CAM(feature_conv, weight_softmax, class_idx):
    """
    Implement CAM here
    refer to: https://github.com/zhoubolei/CAM
    """
    output_cam = []
    ################# Your Implementations #####################################

    ################# End of your Implementations ##############################
    return output_cam

In [ ]:
import matplotlib.pyplot as plt

seed_everything(0)
resnet_finetune3.eval()

finalconv_name = 'layer4'
# hook the feature extractor
features_blobs = []


def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())


resnet_finetune3.resnet._modules.get(finalconv_name).register_forward_hook(
    hook_feature)

params = list(resnet_finetune3.parameters())
weight_softmax = np.squeeze(params[-2].data.cpu().numpy())

classes = []
for i in range(100):
    classes.append(str(i))

val_sub_dataset = torch.utils.data.Subset(
    miniplaces_val, np.random.choice(np.arange(len(miniplaces_val)), 10,
                                     False))
val_sub_loader = torch.utils.data.DataLoader(val_sub_dataset,
                                             batch_size=1,
                                             num_workers=num_workers,
                                             shuffle=False)

figure = plt.figure(figsize=(24, 6))
cols, rows = 10, 2
counter = 0

for i, data in enumerate(tqdm((val_sub_loader))):
    features_blobs = []
    img, label = data
    img = img.cuda()

    logit = resnet_finetune3(img)

    h_x = F.softmax(logit, dim=1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.cpu().numpy()
    idx = idx.cpu().numpy()

    # generate class activation mapping for the top1 prediction
    CAMs = CAM(features_blobs[0], weight_softmax, [idx[0]])

    img = img.squeeze().permute(1, 2, 0)
    height, width, _ = img.shape
    img = img.cpu() * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor(
        [0.485, 0.456, 0.406])
    img = img.cpu().numpy()
    heatmap = cv2.applyColorMap(cv2.resize(CAMs[0], (width, height)),
                                cv2.COLORMAP_JET)
    heatmap = heatmap / 255
    heatmap = heatmap[..., ::-1]  # reverse the color from BGR to RGB
    result = heatmap * 0.7 + img * 0.3

    figure.add_subplot(rows, cols, i + 1)
    plt.axis("off")
    plt.title(
        f"gt: {miniplaces_val.label_dict[label.item()]}\n prob: {h_x[label.item()].item()*100:.2f}"
    )
    plt.imshow(img, cmap="gray")

    figure.add_subplot(rows, cols, i + 11)
    plt.axis("off")
    plt.title(
        f"pred: {miniplaces_val.label_dict[idx[0]]}\n prob: {probs[0]*100:.2f}"
    )

    plt.imshow(result, cmap="gray")
    counter += 1

plt.show()

### Q2.4 Answer the following questions (15 pts)

a: Compare the results of using MLPs in Q3 of Assignment 1, FastConv in Q1, and ResNet in Q2. In 2-3 sentences, explain the differences in these model architectures and how such differences leads to the performance gap between these models. (5 pts)

(Double click to edit)

Your Answer:

---

b: Compare the results of Q2.1, Q2.2.2, Q2.2.3, Q2.2.4, and Q2.2.5? Why finetuning performs better than training from scratch on ResNet-18? Which finetuning techniques gives you the best accuracy? Which finetuning techniques gives you the worst accuracy? Use one sentence to explain the accuracy ordering for each finetuning method. (5 pts)

(Double click to edit)

Your Answer:

---

c: In 2-3 sentences, describe what you observe from the class activation mapping in Q2.3, where is the CNN "looking at" in those images? Give 1-2 examples from the visualizations. (5 pts)

(Double click to edit)

Your Answer:

---

## Q3: Your own model (20 pts) + 10 pts extra credit

In this question, you will train your own CNN on MiniPlaces. You can base your model on the pretrained `ResNet-18` weights, and modify the learning rate, optimizer, and other parameters during finetuning. You may also want to implement other training techniques such as data augmentation. But feel free to try other CNN architectures. You should write this model in the following code cells.

You will select the best model using the validation set, and then evaluate it on the test set.  You should create a file named "test_UID.json", containing "{id: predicted_label}" pairs. You may also try model assembly or test-time augmentation to further improve the performance.

You may also need to modify the previous MiniPlaces dataset to also load test images from `{root_dir}/images/test`.

We will hold a leaderboard of the test accuracy on [Kaggle](https://www.kaggle.com/t/9cb9ae990d5f4576b5fb207ee9bf39dc). See the instructions on the website for submission to the leaderboard. The **top 20%** of students can receive a bonus of 10 points. Don't worry, you shall receive a full credit for this question as long as your model has better accuracy than the finetuning baseline.

Feel free to organize your code in different ways. In this question, you do not have to follow the `#Your Implementations#` comments. Below is just an example of one possible organization for your code.

**Your code should be trainable on Colab**, we do not expect you to use additional resources to train your model, and your code should give similar test accuracies to the ones you posted on the leaderboard.

In [ ]:
# Define your model.

In [ ]:
# Build train/val/test datasets and data loaders using the modules we built before.
import os

from torch.utils.data import DataLoader, Dataset


class MiniPlacesTest(Dataset):

    def __init__(self, root_dir, transform):
        """
        Initialize the MiniPlaces test dataset with the root directory for the images,
        an optional data transformation,


        Args:
            root_dir (str): Root directory for the MiniPlaces images.
            transform (callable, optional): Optional data transformation to apply to the images.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.filenames = []
        self.ids = []

        # You should read the test images from '${root_dir}/images/test' dir and save their filepaths and ids
        # For instance: the id of image 'test/00000001.jpg' should be '00000001' in string
        ################# Your Implementations #################################

        ################# End of your Implementations ##########################

    def __len__(self):
        """
        Return the number of images in the dataset.

        Returns:
            int: Number of images in the dataset.
        """
        dataset_len = 0
        ################# Your Implementations #################################
        # Return the number of images in the dataset

        ################# End of your Implementations ##########################
        return dataset_len

    def __getitem__(self, idx):
        """
        Return a single image and its corresponding id when given an index.

        Args:
            idx (int): Index of the image to retrieve.

        Returns:
            tuple: Tuple containing the image and its id.
        """
        ################# Your Implementations #################################
        # Load and preprocess image using self.root_dir,
        # self.filenames[idx], and self.transform (if specified)

        ################# End of your Implementations ##########################
        return image, id


data_root = os.path.join(root_dir, 'data')
# Create MiniPlaces test dataset object
miniplaces_test = MiniPlacesTest(data_root, transform=data_transform)

# Create DataLoader for test set
test_loader = DataLoader(miniplaces_test,
                         batch_size=batch_size,
                         num_workers=num_workers,
                         shuffle=False)

In [ ]:
# Train on the training set, validate on the validation set.
# Search hyper-parameters to find the best model on the validation set.

In [ ]:
# Generate test-set results.


def test(model, test_loader):
    """
    Evaluate the MLP classifier on the test set.

    Args:
        model (MLP): MLP classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.

    Returns:
        out_dict: prediction result on the test set
    """
    model.eval()  # Set model to evaluation mode
    out_dict = {}

    with torch.no_grad():
        for inputs, ids in test_loader:
            ################# Your Implementations #############################
            # similar to the evaluate function
            # first compute the output logits and then get the predicted class with maximum probability
            # save the id and the prediction to out_dict



            ################# End of your Implementations ######################
    return out_dict

In [ ]:
# Convert the test-set results dictionary to a CSV file:

import csv

test_output = test(resnet_finetune2, test_loader)
assert len(test_output) == 10000
# update your csv file name for submissioin
csv_name = "test.csv"

# now we will open a file for writing
data_file = open(os.path.join(root_dir, csv_name), 'w')

# create the csv writer object
csv_writer = csv.writer(data_file)

# Counter variable used for writing
# headers to the CSV file
count = 0

header = ['Id', 'Category']

for key, value in test_output.items():
    if count == 0:
        # Writing headers of CSV file
        header = header
        csv_writer.writerow(header)
        count += 1

    # Writing data of CSV file
    csv_writer.writerow([key, str(value)])

data_file.close()
print("succesfully saved the test csv file at: " +
      os.path.join(root_dir, csv_name))

from google.colab import files

files.download(os.path.join(root_dir, csv_name))

Please report your test accuracy on the public leaderboard:

(Double click to edit)

Your Answer:

---

In one paragraph, shortly summarize your method and your improvement and modifications made upon the baseline:

(Double click to edit)

Your Answer:

---